# 使用单层全连接CNN识别MNIST

In [1]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

In [2]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001

train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False,
)

train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)/255.   
test_y = test_data.test_labels

D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
D:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2,),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.out = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output, x

cnn = CNN()
print(cnn)

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=3136, out_features=10, bias=True)
)


In [4]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        output = cnn(b_x)[0]
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.3166 | test accuracy: 0.13
Epoch:  0 | train loss: 0.6115 | test accuracy: 0.86
Epoch:  0 | train loss: 0.2769 | test accuracy: 0.89
Epoch:  0 | train loss: 0.1437 | test accuracy: 0.92
Epoch:  0 | train loss: 0.2921 | test accuracy: 0.92
Epoch:  0 | train loss: 0.3752 | test accuracy: 0.94
Epoch:  0 | train loss: 0.3252 | test accuracy: 0.94
Epoch:  0 | train loss: 0.2362 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2158 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2290 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1188 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1814 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1865 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1142 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1541 | test accuracy: 0.97
Epoch:  0 | train loss: 0.2218 | test accuracy: 0.97
Epoch:  0 | train loss: 0.2075 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0808 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1553 | test accuracy

In [5]:
test_output, _ = cnn(test_x[:20])
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(test_y[:20].numpy(), 'real number')

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4] prediction number
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4] real number
